In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110310  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,2.328307e+06
2018-02-28,2.084520e+06
2018-03-31,2.490431e+06
2018-04-30,2.728097e+06
2018-05-31,2.620466e+06
2018-06-30,2.650448e+06
2018-07-31,2.762193e+06
2018-08-31,2.984600e+06
2018-09-30,2.827547e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    2.328307e+06
 2018-02-28    2.084520e+06
 2018-03-31    2.490431e+06
 2018-04-30    2.728097e+06
 2018-05-31    2.620466e+06
 2018-06-30    2.650448e+06
 2018-07-31    2.762193e+06
 2018-08-31    2.984600e+06
 2018-09-30    2.827547e+06
 2018-10-31    2.602857e+06
 2018-11-30    2.341137e+06
 2018-12-31    2.260118e+06
 2019-01-31    2.530546e+06
 2019-02-28    2.341279e+06
 2019-03-31    2.528881e+06
 2019-04-30    2.725538e+06
 2019-05-31    2.888620e+06
 2019-06-30    2.755108e+06
 2019-07-31    2.953160e+06
 2019-08-31    3.207563e+06
 2019-09-30    2.660220e+06
 2019-10-31    2.886933e+06
 2019-11-30    2.514243e+06
 2019-12-31    2.535724e+06
 2020-01-31    2.742946e+06
 2020-02-29    2.153948e+06
 2020-03-31    2.416371e+06
 2020-04-30    2.361057e+06
 2020-05-31    2.799001e+06
 2020-06-30    2.599885e+06
 2020-07-31    2.891116e+06
 2020-08-31    3.295605e+06
 2020-09-30    2.986161e+06
 2020-10-31    3.292055e+06
 2020-11-30    2.917515e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : 0.685656
p-value : 0.989547
Critical Values :
	1%: -3.646135
	5%: -2.954127
	10%: -2.615968


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.765005
p-value : 0.000063
Critical Values :
	1%: -3.646135
	5%: -2.954127
	10%: -2.615968


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=1144.365, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=1143.407, Time=0.09 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=1142.016, Time=0.05 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=1144.029, Time=0.06 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=1145.142, Time=0.09 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=1146.849, Time=0.37 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=1142.526, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=1144.184, Time=0.06 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=1143.723, Time=0.10 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=1143.397, Time=0.16 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=1144.835, Time=0.17 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=1143.046, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=1144.585, Time=0.09 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=1143.561, Time=0.22 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.52 sec
 ARIMA(3,1,0)(0

ARIMA(order=(0, 1, 2), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(0,1,2))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(0, 1, 2)   Log Likelihood                -568.380
Method:                       css-mle   S.D. of innovations         253398.356
Date:                Sat, 11 Sep 2021   AIC                           1144.759
Time:                        01:45:52   BIC                           1151.614
Sample:                             1   HQIC                          1147.255
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.639e+04   3.55e+04      0.743      0.458   -4.32e+04     9.6e+04
ma.L1.D.y     -0.2618      0.150     -1.747      0.081      -0.556       0.032
ma.L2.D.y      0.1589      0.190      0.838      0.4

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([3504486.55682354, 3530872.59844484]), array([253398.35588148, 314956.37246499]), array([[3007834.90555417, 4001138.20809292],
       [2913569.45171207, 4148175.74517762]]))
